Algorithm - **Sparse Classification**  
Preprocessing - **No preprocessing**  
Feature - **Magnitude**


In [ ]:
# Library
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import DictionaryLearning, sparse_encode
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# Load data
measurement = np.load('../../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:4000]  # Using 1000 samples for simplicity

# Split data
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6    # Channel 6 is EVE (illegitimate)

In [53]:
# ----------------------------------------------------- Preprocessing -----------------------------------------------------
# ----------------- Training data -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs

# amplitude
# np.clip() to ensure that the input to np.sqrt() is non-negative. This will replace any negative values with 0.
# alice_train = np.sqrt(np.clip(alice_train_CIRs[..., 0] ** 2 + alice_train_CIRs[..., 1] ** 2, a_min=0, a_max=None))
train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])

# Traning Labels
train_alice_labels = np.zeros(train_alice_magnitude.shape[0])  # Label '0' for Alice.
train_eve_labels = np.ones(train_eve_magnitude.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((train_alice_labels, train_eve_labels))

# Atoms
train_atoms = np.vstack((train_alice_magnitude, train_eve_magnitude))

print('Training data shape:', train_atoms.shape)

# Dictionary
D = train_atoms.T
print('Dictionary shape:', D.shape)


# ----------------- Testing data -----------------
test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]

# test amplitude
test_alice_magnitude = np.abs(test_alice_CIRs[..., 0] + 1j * test_alice_CIRs[..., 1])
test_eve_magnitude = np.abs(test_eve_CIRs[..., 0] + 1j * test_eve_CIRs[..., 1]) 
test_cirs = np.vstack((test_alice_magnitude, test_eve_magnitude))

# labels
test_alice_labels = np.zeros(test_alice_magnitude.shape[0])  # Label '0' for Alice.
test_eve_labels = np.ones(test_eve_magnitude.shape[0])    # Label '1' for Eve.
test_labels = np.hstack((test_alice_labels, test_eve_labels))


Training data shape: (6400, 251)
Dictionary shape: (251, 6400)


In [54]:
# ---------------------------------------------- Sparse Coding & Classification ------------------------------------------------
# Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=5):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# Calculate Residuals for Each Class
def calculate_residual(tSample, D, coefficients, class_indices):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]
    
    reconstructed_signal = D @ coef_class # (251,) = (251, 1600) @ (1600,)
    residual = np.linalg.norm(tSample - reconstructed_signal)

    return residual

# Classification Function
def classify_signal(tSample, D, trainLabel):
    
    # sparse coefficients for the new signal
    coefficients = find_sparse_coefficients(tSample, D) 
    # coefficients.shape = (number_of_atoms/signal, )
    
    # Residuals list
    residuals = []

    # Calculate residual for each class
    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)  # Indices of columns in D belonging to this class
        # print('class_indices:', class_indices)
        residual = calculate_residual(tSample, D, coefficients, class_indices)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    # print('Predicted class:', predicted_class)
    return predicted_class

# Step 8: Classifying Test Data and Evaluating the Model
predictions = []

for cir in test_cirs:
    predicted_class = classify_signal(cir, D, train_labels)
    predictions.append(predicted_class)

predictions = np.array(predictions)
# print(predictions.shape)


In [55]:
# ---------------------------------------------- Evaluation ------------------------------------------------
accuracy = np.mean(predictions == test_labels)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
print(f"\nTotal testing channel: {test_labels.shape}")

tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# # Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# # False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# # Gamma calculation
gamma = (tp + fn) / (tn + fp)

# # Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")

Classification Accuracy: 60.88%

Total testing channel: (1600,)
tp: 678
tn: 296
fp: 504
fn: 122
MDR: 0.63
FAR: 0.1525
AR: 0.60875
